In [1]:
import duckdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

2025-01-31 23:45:53.583132: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738341953.598439   30777 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738341953.603023   30777 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-31 23:45:53.623259: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Load dataset
con = duckdb.connect(database=":memory:")
df = con.read_parquet("data/pm25*.parquet").df()

# Add 'day' and 'hour' columns from the timestamp
df.insert(loc=3, column="day", value=df["timestamp"].dt.day)
df.insert(loc=4, column="hour", value=df["timestamp"].dt.hour)

# Remove timezone info from the timestamp
df["timestamp"] = df["timestamp"].dt.tz_localize(None)

# Preprocessing
# Assuming we have features like 'temperature', 'humidity', 'wind_speed', etc.
features = [
    "year",
    "month",
    "day",
    "hour",
    "temperature",
    "humidity",
    "pressure",
    "wind_direction",
    "wind_speed",
    "pm2_5",
]
df = df[features + ["pollution_level"]].copy()

# Mapping pollution levels to numerical values
pollution_level_mapping = {
    "คุณภาพอากาศดีมาก": 0,  # Very Good
    "คุณภาพอากาศดี": 1,     # Good
    "ปานกลาง": 2,            # Moderate
    "เริ่มมีผลกระทบต่อสุขภาพ": 3,  # Unhealthy for Sensitive Groups
    "มีผลกระทบต่อสุขภาพ": 4    # Unhealthy
}

# Apply mapping to pollution_level
df['pollution_level_class'] = df['pollution_level'].map(pollution_level_mapping)

# Sort the DataFrame
df = df.sort_values(by=['year', 'month', 'day', 'hour'], ascending=True)
# prompt: keep only one record for each row

# Check for duplicates based on all columns except 'pollution_level'
duplicates = df[df.duplicated(subset=features, keep=False)]

# If duplicates exist, keep only the first occurrence
if not duplicates.empty:
    df = df.drop_duplicates(subset=features, keep='first')

# Filter the data into train and test based on year
train_data = df[df['year'].isin([2021, 2022, 2023])]
test_data = df[df['year'] == 2024]
#test_data = df[df['year'].isin([2023, 2024])]

# Scaling features except for the target ('pollution_level_class')
#scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train_features = scaler.fit_transform(train_data[features])
scaled_test_features = scaler.transform(test_data[features])

# Encode target labels to categorical
y_train_encoded = to_categorical(train_data['pollution_level_class'])
y_test_encoded = to_categorical(test_data['pollution_level_class'])

# Adjust the sequence creation function for 12-hour prediction
def create_sequences(data, target, look_back, predict_ahead):
    X, y = [], []
    for i in range(48,len(data) - look_back - predict_ahead + 1):
        X.append(data[i:i + look_back, :])  # Use all features for past `look_back`
        y.append(target[i + look_back:i + look_back + predict_ahead])  # Predict `predict_ahead` steps
    return np.array(X), np.array(y)

look_back = 48  # Past 20 time steps
predict_ahead = 12  # Predict the next 12 hours

# Create sequences
X_train, y_train = create_sequences(scaled_train_features, y_train_encoded, look_back, predict_ahead)
X_test, y_test = create_sequences(scaled_test_features, y_test_encoded, look_back, predict_ahead)
# X_train, y_train = create_sequences(train_data[features].values, y_train_encoded, look_back, predict_ahead)
# X_test, y_test = create_sequences(train_data[features].values, y_test_encoded, look_back, predict_ahead)

In [ ]:
# Define the focal loss function
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        # Clip y_pred to avoid log(0) error
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        # Calculate focal loss for each time step using one-hot encoded labels
        # tf.where requires y_true to have the same shape as y_pred

        # Calculate cross-entropy loss
        ce_loss = -y_true * K.log(y_pred)

        # Apply focal weights
        loss = alpha * K.pow(1. - y_pred, gamma) * ce_loss

        # Modification: Sum over the last axis to get a single loss value
        loss = K.sum(loss, axis=-1)

        # Average loss over the time dimension
        return K.mean(loss)  # Calculate the mean over the remaining dimensions
    return focal_loss_fixed

In [ ]:
# prompt: add focal loss to lstm model and add precision and recall matrix

from tensorflow.keras import backend as K
import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint

# Define LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=y_train.shape[1] * y_train.shape[2]))
model.add(Dropout(0.2))
model.add(Reshape((y_train.shape[1], y_train.shape[2])))
model.add(Dense(units=y_train.shape[2], activation='softmax'))  # Output layer with softmax

# Compile the model with focal loss
model.compile(optimizer='adam',
              loss=focal_loss(alpha=0.25, gamma=2),
              #loss='categorical_crossentropy',
              metrics=['accuracy', 'Precision', 'Recall'])

# Add a model checkpoint callback
checkpoint_filepath = 'best_model.keras'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)


model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,
    batch_size=32,
    callbacks=[model_checkpoint_callback]
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Assuming 'history' is the output of model.fit()
# Plotting training and validation loss
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
# ... (rest of your model training code) ...

from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load the saved best model
model = load_model(
    './best_model.keras',
    custom_objects={'focal_loss_fixed': focal_loss(alpha=0.25, gamma=2)}  # Add custom objects if required
)

y_test_classes = np.argmax(y_test, axis=2) + 1
# Assuming y_test and y_pred are of shape (samples, timesteps, classes)
y_pred = model.predict(X_test)
# y_pred_classes = (y_pred > 0.4).astype(int)  # Apply threshold (adjust if needed)
y_pred_classes = np.argmax(y_pred, axis=2) + 1

# Reshape to (samples * timesteps, classes) for sklearn metrics
# y_test_flat = y_test.reshape(-1, y_test.shape[-1])
# y_pred_flat = y_pred_classes.reshape(-1, y_pred_classes.shape[-1])

# # Calculate and print the classification report
# print(classification_report(y_test_flat, y_pred_flat))

# # Calculate the multilabel confusion matrix
# cm = multilabel_confusion_matrix(y_test_flat, y_pred_flat)

# # Plot confusion matrices for each label
# num_labels = y_test.shape[-1]  # Get the number of labels
# fig, axes = plt.subplots(num_labels, 1, figsize=(8, 6 * num_labels))

# for i in range(num_labels):
#     sns.heatmap(cm[i], annot=True, fmt="d", cmap="Blues", ax=axes[i])
#     axes[i].set_xlabel('Predicted')
#     axes[i].set_ylabel('True')
#     axes[i].set_title(f'Confusion Matrix for Label {i}')

# plt.tight_layout()  # Adjust spacing between subplots
# plt.show()

In [ ]:
# prompt: Calculate MAPE using y_test_classes and y_pred_classes

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test_classes, y_pred_classes)
print(f"MAPE: {mape:.2f}%")

In [ ]:
# prompt: Calculate Recall, Precision, F1, accuracy using y_test_classes and y_pred_classes

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming y_test_classes and y_pred_classes are defined as in your provided code

# Flatten the arrays for sklearn metrics if they are not already flattened
y_test_classes_flat = y_test_classes.flatten()
y_pred_classes_flat = y_pred_classes.flatten()

# Calculate the metrics
accuracy = accuracy_score(y_test_classes_flat, y_pred_classes_flat)
precision = precision_score(y_test_classes_flat, y_pred_classes_flat, average='macro') # Use macro averaging for multi-class
recall = recall_score(y_test_classes_flat, y_pred_classes_flat, average='macro') # Use macro averaging for multi-class
f1 = f1_score(y_test_classes_flat, y_pred_classes_flat, average='macro') # Use macro averaging for multi-class

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")